In [2]:
# Imports and Consts
SEED = 42 # For determnistic testing
import numpy as np
from util import get_data

/Users/tudor/CS-Work/ML/P1/LogisticRegression/LogisticRegression/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Read data and construct data sets
data = get_data()
data = data[1:]
data = data.astype(float)
xs = data[:, :-1]
ys = data[:, -1]
print(xs.shape)

# Add column to account for const B value in training
ones = [[1.0]] * len(xs)
xs = np.append(xs, ones, axis=1)

(768, 8)
(768, 9)


In [24]:
# Normalize Data
xs = xs / np.linalg.norm(xs, axis=0, keepdims=True)
xs = (xs - np.mean(xs, axis=0)) / np.std(xs, axis=0)

# Shuffle
np.random.seed(42)
idxs = np.random.permutation(len(xs))

xs = xs[idxs]
ys = ys[idxs]

# Split Data
x_train = xs[:614]
y_train = ys[:614]
x_dev = xs[614:691]
y_dev = ys[614:691]
x_test = xs[691:]
y_test = ys[691:]


In [ ]:
# Helpers

def Pr(arr: np.ndarray, param: np.ndarray) -> np.ndarray:
    exp = np.exp(arr @ param)
    return exp / (1 + exp)

def construct_w(p: np.ndarray) -> np.ndarray:
    return np.diag((1 - p) * p)

def compute_delta(X: np.ndarray, diag: np.ndarray, Y: np.ndarray, p: np.ndarray) -> np.ndarray:
    H = X.T @ diag @ X
    B = X.T @ (Y - p)

    # Speed up computation via decomposition
    L = np.linalg.cholesky(H)
    
    # Solve first only with L
    temp = np.linalg.solve(L, B)

    # Return final solution by solving for temp with L.T
    return np.linalg.solve(L.T, temp)

(9,)

In [ ]:
def newton_descent(X):
    # Initialize Theta
    np.random.seed(42)
    theta = np.random.randn(xs.shape[1])